In [1]:
import sys
sys.path.append("..")  # Add the project root to Python path
from app.controller.label_classifier import LabelClassifier

classifer = LabelClassifier()

In [ ]:
classifer.label_tree.light_tree

In [2]:
task_id = 'f5d131fb-e531-4401-a22a-9f1f6f1c2e04'
task_goal = 'The user is experiencing performance issues when executing a simple SELECT * FROM table_name LIMIT 1 query on a large table in TiDB. The execution plan indicates that the query takes over 20 seconds to complete, despite having sufficient memory and CPU resources. The table has been configured with TTL (Time to Live), which might be affecting performance. The user is using TiDB version 8.1 in a POC environment. 8000w的表不加order where条件limit 1要20多秒 【 TiDB 使用环境】Poc 【 TiDB 版本】V8.1 【复现路径】对应的表加过ttl 【遇到的问题：问题现象及影响】 sql select * from table_name limit 1 执行计划 | id | estRows | estCost | actRows | task | access object | execution info | operator info | memory | disk | | Limit_7 | 1.00 | 67.35 | 0 | root | | time:29.4s, loops:1 | offset:0, count:1 | N/A | N/A | | └─TableReader_11 | 1.00 | 67.35 | 0 | root | | time:29.4s, loops:1, cop_task: {num: 794, max: 472.2ms, min: 485.7µs, avg: 36.8ms, p95: 231.7ms, tot_proc: 28.5s, tot_wait: 251.7ms, copr_cache_hit_ratio: 0.78, build_task_duration: 705.1ms, max_distsql_concurrency: 1}, rpc_info:{Cop:{num_rpc:794, total_time:29.2s}} | data:Limit_10 | 339 Bytes | N/A | | └─Limit_10 | 1.00 | 305.49 | 0 | cop[tikv] | | tikv_task:{proc max:471ms, min:0s, avg: 114ms, p80:147ms, p95:260ms, iters:794, tasks:794}, scan_detail: {total_keys: 86977944, get_snapshot_time: 230.8ms, rocksdb: {delete_skipped_count: 6326304, key_skipped_count: 93304074, block: {cache_hit_count: 2379, read_count: 150700, read_byte: 1.03 GB, read_time: 4.27s}}}, time_detail: {total_process_time: 28.5s, total_wait_time: 251.7ms, total_kv_read_wall_time: 28.4s, tikv_wall_time: 28.9s} | offset:0, count:1 | N/A | N/A | | └─TableFullScan_9 | 1.00 | 305.49 | 0 | cop[tikv] | table:data_event_relation | tikv_task:{proc max:471ms, min:0s, avg: 114ms, p80:147ms, p95:260ms, iters:794, tasks:794} | keep order:false | N/A | N/A | 【资源配置】 内存cpu充足'
label_path = classifer.generate_label_description(task_goal)
label_path

[{'label': 'Troubleshooting',
  'description': 'Diagnostic guidance and problem-solving approaches for system issues, error conditions, or unexpected behaviors.'},
 {'label': 'SQL Optimization',
  'description': 'Strategies, best practices, and techniques for enhancing the performance and efficiency of SQL queries.'}]

In [ ]:
label_path = [
  {'label': 'Troubleshooting',
  'description': 'Diagnostic guidance and problem-solving approaches for system issues, error conditions, or unexpected behaviors. Focuses on root cause analysis and resolution strategies for common operational challenges.'},
 {'label': 'SQL Optimization',
   'description': 'Strategies, best practices, and techniques for enhancing the performance and efficiency of SQL queries. Focuses on query tuning, indexing, execution plan analysis, and reducing resource consumption in database systems to ensure scalability and responsiveness.'}
]
label_path

In [3]:
classifer.insert_label_path(task_id, label_path)

In [ ]:
import requests

def save_best_plan_from_url(url):
    
    try:
        # Split URL by '/' and extract components
        parts = url.split('/')
        
        # Find index of 'tasks' and extract task_id and commit_hash
        if 'tasks' in parts:
            tasks_index = parts.index('tasks')
            if len(parts) < tasks_index + 4:  # Ensure we have enough parts after 'tasks'
                raise ValueError("Invalid URL format")
                
            task_id = parts[tasks_index + 1]
            commit_hash = parts[tasks_index + 3]

            url = f"https://stackvm.tidb.ai/api/tasks/{task_id}/commits/{commit_hash}/save_best_plan"
            response = requests.post(url)
            if response.status_code != 200:
                raise ValueError("Failed to save best plan")
            
    except Exception as e:
        print(e)
        return False
    
    return True

url = "https://stackvm-ui.vercel.app/tasks/f5d131fb-e531-4401-a22a-9f1f6f1c2e04/plan_update_20250110_052226/3ef8db3817434ec0b5843edfa67227a5/final-answer"
save_best_plan_from_url(url)


In [ ]:
labels = [
    {
        "label": "Troubleshooting",
        "description": "Diagnostic guidance and problem-solving approaches for system issues, error conditions, or unexpected behaviors."
    }
]

matching_node = classifer.label_tree.find_longest_matching_label(labels)
tasks = classifer.label_tree.get_all_tasks_under_label(matching_node)
tasks

#tasks_plan = get_task_plans([task["id"] for task in tasks])
#tasks_plan



In [6]:
from typing import Dict, Any, List

def format_tasks_plans(
    node: Dict[str, Any],
    current_path: List[str] = None
) -> str:
    """
    Traverses the tree structure and formats each task into a specified text format.

    Parameters:
    - node (Dict[str, Any]): The current node in the tree.
    - task_plans (Dict[str, Dict[str, Any]]): A dictionary mapping task IDs to their best plans.
    - current_path (List[str], optional): The path of labels from the root to the current node.

    Returns:
    - str: The formatted text containing all tasks.
    """
    if current_path is None:
        current_path = []

    output = []

    # Update the current path with the current node's name
    node_name = node.get("name", "")
    new_path = current_path + [node_name] if node_name else current_path

    # Process tasks in the current node
    tasks = node.get("tasks", [])
    for task in tasks:
        task_id = task.get("id")
        task_goal = task.get("goal")
        best_plan = task.get("best_plan",  None)
        if best_plan is None:
            continue

        task_text = (
            f"task: {task_goal}\n"
            f"label_path: {' -> '.join(new_path)}\n"
            f"best_plan: {best_plan}\n"
            "-------\n"
        )
        output.append(task_text)

    # Recursively process child nodes
    children = node.get("children", [])
    for child in children:
        output.append(format_tasks_plans(child, new_path))

    return ''.join(output)

In [ ]:
from app.services.llm_interface import LLMInterface
from app.config.settings import LLM_PROVIDER, FAST_LLM_MODEL
from app.services import get_best_pratices_prompt


formatted_task_plan =  format_tasks_plans(matching_node)
label_path = ' -> '.join([label["label"] for label in labels])
prompt = get_best_pratices_prompt(label_path, formatted_task_plan)

best_pratices_str = LLMInterface(LLM_PROVIDER, FAST_LLM_MODEL).generate(prompt)
print(best_pratices_str)

In [10]:
best_practices_str = """To solve tasks in the <Troubleshooting> category, please follow this approach:
1. Problem Analysis and Hypothesis Generation
  - Clearly define the scope and impact of the issue during the initial reasoning phase (Problem Framing).
  - Generate multiple hypotheses or potential causes (Hypothesis Generation) to avoid narrowing the investigation too early.
  - This step ensures a thorough understanding of the problem and provides a roadmap for subsequent research and analysis.

  The example of chain_of_thoughts: "The main issue stems from the concentrated write operations due to a date prefix index, resulting in an index hotspot. Optimizing the index by replacing the date prefix could help scatter writes more evenly, potentially using attributes like user ID for better distribution. Moreover, partitioning tables can be a robust method to further distribute writes across multiple regions, spreading out the load and addressing the hotspot problem effectively. This approach involves both adjusting the index structure and implementing partitioned tables as a combined strategy.",


2. Multi-Channel Information Gathering and Cross-Verification
  - Leverage a Knowledge Graph for structured insights and connections related to the issue.
  - Use Vector Search to locate similar cases or documentation based on content similarity.
  - Ttailor your queries/prompts to address specific error points rather than general issues/topics. This ensures that both the data collected and the subsequent analysis are closely aligned with the problem at hand.
  -	Employ LLM-based summarization and inference to synthesize findings from various sources.
  - By integrating different sources of information, you gain a more comprehensive understanding of the root causes and possible solutions.

3.	Reasoning and Solution Generation (With Irrelevant Solution Filtering)

  - During the reasoning phase, evaluate the collected information against your initial hypotheses.
  - Filter out any solutions or suggestions that do not directly address the identified problem to ensure relevancy.
  - Consolidate valid insights into clear, actionable recommendations.
  - This ensures that the final solutions are both targeted and feasible for resolving the specific issue at hand.

By following these steps—starting with thorough problem analysis, then gathering and verifying data from multiple sources, and finally synthesizing a well-founded solution while filtering out irrelevant approaches—you can enhance the accuracy and efficiency of your troubleshooting efforts.
"""


In [11]:
from app.models.label import Label
from app.database import SessionLocal

with SessionLocal() as session:
    label = session.query(Label).filter(Label.name == labels[-1].get("label")).first()
    label.best_practices = best_practices_str
    session.add(label)
    session.commit()